# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 10 2023 8:30AM,259043,10,PRF-55141,Bio-PRF,Released
1,Mar 10 2023 8:30AM,259043,10,PRF-55148,Bio-PRF,Released
2,Mar 10 2023 8:30AM,259043,10,PRF-55155,Bio-PRF,Released
3,Mar 10 2023 8:30AM,259043,10,PRF-55159,Bio-PRF,Released
4,Mar 9 2023 3:29PM,259050,10,MSP219367,Methapharm-G,Released
5,Mar 9 2023 3:29PM,259050,10,MSP219366,"Methapharm, Inc.",Released
6,Mar 9 2023 3:29PM,259050,10,MSP219365,"Methapharm, Inc.",Released
7,Mar 9 2023 3:26PM,259048,19,TR-NR-03102023,"GCH Granules USA, Inc.",Released
8,Mar 9 2023 3:23PM,259047,16,9248720,Sartorius Bioprocess Solutions,Released
9,Mar 9 2023 3:22PM,259046,16,SSF_C1W11,Sartorius Stedim Filters Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,259045,Released,1
24,259046,Released,1
25,259047,Released,1
26,259048,Released,1
27,259050,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259045,NaN,NaN,1.0
259046,NaN,NaN,1.0
259047,NaN,NaN,1.0
259048,NaN,NaN,1.0
259050,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258979,0.0,0.0,1.0
258980,48.0,4.0,5.0
258983,14.0,19.0,0.0
258993,0.0,0.0,1.0
258995,2.0,13.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258979,0,0,1
258980,48,4,5
258983,14,19,0
258993,0,0,1
258995,2,13,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258979,0,0,1
1,258980,48,4,5
2,258983,14,19,0
3,258993,0,0,1
4,258995,2,13,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258979,,,1
1,258980,48,4,5
2,258983,14,19,
3,258993,,,1
4,258995,2,13,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 10 2023 8:30AM,259043,10,Bio-PRF
4,Mar 9 2023 3:29PM,259050,10,Methapharm-G
5,Mar 9 2023 3:29PM,259050,10,"Methapharm, Inc."
7,Mar 9 2023 3:26PM,259048,19,"GCH Granules USA, Inc."
8,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions
9,Mar 9 2023 3:22PM,259046,16,Sartorius Stedim Filters Inc.
10,Mar 9 2023 3:09PM,259045,12,"SD Head USA, LLC"
11,Mar 9 2023 3:07PM,259044,19,"GCH Granules USA, Inc."
12,Mar 9 2023 2:50PM,259042,19,Eli Lilly and Company
13,Mar 9 2023 2:47PM,259041,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 10 2023 8:30AM,259043,10,Bio-PRF,,,4
1,Mar 9 2023 3:29PM,259050,10,Methapharm-G,,,3
2,Mar 9 2023 3:29PM,259050,10,"Methapharm, Inc.",,,3
3,Mar 9 2023 3:26PM,259048,19,"GCH Granules USA, Inc.",,,1
4,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions,,,1
5,Mar 9 2023 3:22PM,259046,16,Sartorius Stedim Filters Inc.,,,1
6,Mar 9 2023 3:09PM,259045,12,"SD Head USA, LLC",,,1
7,Mar 9 2023 3:07PM,259044,19,"GCH Granules USA, Inc.",,,1
8,Mar 9 2023 2:50PM,259042,19,Eli Lilly and Company,,,1
9,Mar 9 2023 2:47PM,259041,10,"Snap Medical Industries, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:30AM,259043,10,Bio-PRF,4,,
1,Mar 9 2023 3:29PM,259050,10,Methapharm-G,3,,
2,Mar 9 2023 3:29PM,259050,10,"Methapharm, Inc.",3,,
3,Mar 9 2023 3:26PM,259048,19,"GCH Granules USA, Inc.",1,,
4,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions,1,,
5,Mar 9 2023 3:22PM,259046,16,Sartorius Stedim Filters Inc.,1,,
6,Mar 9 2023 3:09PM,259045,12,"SD Head USA, LLC",1,,
7,Mar 9 2023 3:07PM,259044,19,"GCH Granules USA, Inc.",1,,
8,Mar 9 2023 2:50PM,259042,19,Eli Lilly and Company,1,,
9,Mar 9 2023 2:47PM,259041,10,"Snap Medical Industries, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:30AM,259043,10,Bio-PRF,4,,
1,Mar 9 2023 3:29PM,259050,10,Methapharm-G,3,,
2,Mar 9 2023 3:29PM,259050,10,"Methapharm, Inc.",3,,
3,Mar 9 2023 3:26PM,259048,19,"GCH Granules USA, Inc.",1,,
4,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:30AM,259043,10,Bio-PRF,4.0,NaN,NaN
1,Mar 9 2023 3:29PM,259050,10,Methapharm-G,3.0,NaN,NaN
2,Mar 9 2023 3:29PM,259050,10,"Methapharm, Inc.",3.0,NaN,NaN
3,Mar 9 2023 3:26PM,259048,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:30AM,259043,10,Bio-PRF,4.0,0.0,0.0
1,Mar 9 2023 3:29PM,259050,10,Methapharm-G,3.0,0.0,0.0
2,Mar 9 2023 3:29PM,259050,10,"Methapharm, Inc.",3.0,0.0,0.0
3,Mar 9 2023 3:26PM,259048,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2590261,16.0,13.0,2.0
12,777082,2.0,1.0,0.0
15,258997,43.0,0.0,0.0
16,518093,2.0,0.0,0.0
19,1813143,9.0,24.0,62.0
21,259020,1.0,0.0,0.0
22,258993,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2590261,16.0,13.0,2.0
1,12,777082,2.0,1.0,0.0
2,15,258997,43.0,0.0,0.0
3,16,518093,2.0,0.0,0.0
4,19,1813143,9.0,24.0,62.0
5,21,259020,1.0,0.0,0.0
6,22,258993,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,16.0,13.0,2.0
1,12,2.0,1.0,0.0
2,15,43.0,0.0,0.0
3,16,2.0,0.0,0.0
4,19,9.0,24.0,62.0
5,21,1.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,16.0
1,12,Released,2.0
2,15,Released,43.0
3,16,Released,2.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,2.0,0.0,0.0,0.0,62.0,0.0,0.0
Executing,13.0,1.0,0.0,0.0,24.0,0.0,0.0
Released,16.0,2.0,43.0,2.0,9.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,2.0,0.0,0.0,0.0,62.0,0.0,0.0
1,Executing,13.0,1.0,0.0,0.0,24.0,0.0,0.0
2,Released,16.0,2.0,43.0,2.0,9.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,2.0,0.0,0.0,0.0,62.0,0.0,0.0
1,Executing,13.0,1.0,0.0,0.0,24.0,0.0,0.0
2,Released,16.0,2.0,43.0,2.0,9.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()